# KLUE - RE(Relation Extraction task)

- Relation Extraction task는 모델이 두 개체의 관계를 이해했는지 평가하기에 적합합니다.
- 데이터는 위키피디아, 위키트리(뉴스 도메인), 정책 브리핑(뉴스 도메인)을 사용합니다.
- Label은 다음과 같습니다.
  - **person-related relations** 18개
  - **organization-related relations** 11개
  - **no_relation**

- 예시는 다음과 같습니다.
  - 문장 : "이순신은 1545년에 태어났다".
  - 출생년도 **관계**로 분류 -> 이순신 - `[출생년도]` - 1545년

# 필요 라이브러리 설치

In [ ]:
!pip install  evaluate
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
#!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is

In [ ]:
!pip install -U transformers datasets==2.21.0 scipy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstall

# 데이터 로딩

In [ ]:
from datasets import load_dataset

task = "re"
raw_datasets = load_dataset("klue", task)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/32470 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7765 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 32470
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source'],
        num_rows: 7765
    })
})

In [ ]:
raw_datasets["train"][0]

{'guid': 'klue-re-v1_train_00000',
 'sentence': '〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.',
 'subject_entity': {'word': '비틀즈',
  'start_idx': 24,
  'end_idx': 26,
  'type': 'ORG'},
 'object_entity': {'word': '조지 해리슨',
  'start_idx': 13,
  'end_idx': 18,
  'type': 'PER'},
 'label': 0,
 'source': 'wikipedia'}

In [ ]:
train_id = []
for i, e in enumerate(raw_datasets["train"]):
    if e["label"] != 0:
        train_id.append(i)

valid_id = []
for i, e in enumerate(raw_datasets["validation"]):
    if e["label"] != 0:
        valid_id.append(i)

In [ ]:
# 훈련, 검증으로 사용할 문장들의 id
train_id[:3], valid_id[:3]

([2, 3, 5], [6, 7, 9])

In [ ]:
# 레이블 확인
relation_feature = raw_datasets["train"].features["label"]
relation_feature

ClassLabel(names=['no_relation', 'org:dissolved', 'org:founded', 'org:place_of_headquarters', 'org:alternate_names', 'org:member_of', 'org:members', 'org:political/religious_affiliation', 'org:product', 'org:founded_by', 'org:top_members/employees', 'org:number_of_employees/members', 'per:date_of_birth', 'per:date_of_death', 'per:place_of_birth', 'per:place_of_death', 'per:place_of_residence', 'per:origin', 'per:employee_of', 'per:schools_attended', 'per:alternate_names', 'per:parents', 'per:children', 'per:siblings', 'per:spouse', 'per:other_family', 'per:colleagues', 'per:product', 'per:religion', 'per:title'], id=None)

In [ ]:
label_names = relation_feature.names
label_names

['no_relation',
 'org:dissolved',
 'org:founded',
 'org:place_of_headquarters',
 'org:alternate_names',
 'org:member_of',
 'org:members',
 'org:political/religious_affiliation',
 'org:product',
 'org:founded_by',
 'org:top_members/employees',
 'org:number_of_employees/members',
 'per:date_of_birth',
 'per:date_of_death',
 'per:place_of_birth',
 'per:place_of_death',
 'per:place_of_residence',
 'per:origin',
 'per:employee_of',
 'per:schools_attended',
 'per:alternate_names',
 'per:parents',
 'per:children',
 'per:siblings',
 'per:spouse',
 'per:other_family',
 'per:colleagues',
 'per:product',
 'per:religion',
 'per:title']

In [ ]:
label_names.remove("no_relation")

In [ ]:
from IPython.display import display, HTML
from datasets import ClassLabel
import random
import pandas as pd


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)

        # 이미 등록된 예제가 뽑힌 경우, 다시 추출
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)

        picks.append(pick)

    # 임의로 추출된 인덱스들로 구성된 데이터 프레임 선언
    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        # 라벨 클래스를 스트링으로 변환
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i] + f"({str(i)})")

    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_train_18294,"현대자동차의 ‘제네시스’가 미국 최고 권위의 내구품질조사(이하 VDS, Vehicle Dependability Study)'에서 조사 대상에 포함된 첫해 1위를 차지하며 글로벌 최고 수준의 품질을 인정받았다.","{'word': '현대자동차', 'start_idx': 0, 'end_idx': 4, 'type': 'ORG'}","{'word': '제네시스', 'start_idx': 8, 'end_idx': 11, 'type': 'POH'}",org:product(8),wikitree
1,klue-re-v1_train_23090,"카를 마르크스(Karl Marx)와 프리드리히 엥겔스(Friedrich Engels)는 포이어바흐의 무신론에 크게 영향을 받았으나, 포이어바흐의 유물론에 대한 모순된 태도에 대해서는 비판하기도 하였다.","{'word': '마르크스', 'start_idx': 3, 'end_idx': 6, 'type': 'PER'}","{'word': '무신론', 'start_idx': 56, 'end_idx': 58, 'type': 'POH'}",per:religion(28),wikipedia
2,klue-re-v1_train_01996,"《가면라이더 W》에서 가면라이더 스컬로 변신하는 나루미 소우키치([[킷카와 코지]]가 연기)는 [[가면라이더 W의 등장 인물 아키코|나루미 아키코]]의 아버지로, 후토에 나루미 탐정 사무소를 연 인물이며, 나루미 탐정 사무소에서 [[가면라이더 더블 (캐릭터) 쇼타로|히다리 쇼타로]]를 제자로 삼아 그를 가르치고 있었다.","{'word': '나루미 소우키치', 'start_idx': 27, 'end_idx': 34, 'type': 'PER'}","{'word': '킷카와 코지', 'start_idx': 38, 'end_idx': 43, 'type': 'PER'}",per:alternate_names(20),wikipedia
3,klue-re-v1_train_17188,"원세개는 중화민국 남경 임시정부의 임시 대총통인 손문(孫文)으로부터 공화정이 수립되면 총통 자리를 넘기겠다는 밀약을 받고, 군벌을 동원하여 선통제의 섭정인 융유황태후(隆裕皇太后)를 압박하였다.","{'word': '원세개', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}","{'word': '선통제', 'start_idx': 78, 'end_idx': 80, 'type': 'PER'}",no_relation(0),wikipedia
4,klue-re-v1_train_16376,크레모니데아 전쟁 직후 안티고노스는 셀레우코스 왕조의 안티오쿠스 2세와 합세하여 공동의 적 프톨레마이오스 2세와 맞섰다.,"{'word': '안티오쿠스 2세', 'start_idx': 30, 'end_idx': 37, 'type': 'PER'}","{'word': '셀레우코스 왕조', 'start_idx': 20, 'end_idx': 27, 'type': 'ORG'}",per:employee_of(18),wikipedia
5,klue-re-v1_train_19270,"코페르니쿠스는 (점성술에 관한 그의 업적은 이론에만 그쳤는데, 경험주의적 천문학은 물론) 점성술을 사용하지 않았지만, 튀코 브라헤와 요하네스 케플러 그리고 갈릴레오 갈릴레이와 같이 아이작 뉴턴 이전의 가장 저명한 천문학자들은 직업이 점성술사였다.","{'word': '튀코 브라헤', 'start_idx': 66, 'end_idx': 71, 'type': 'PER'}","{'word': '요하네스 케플러', 'start_idx': 74, 'end_idx': 81, 'type': 'PER'}",no_relation(0),wikipedia
6,klue-re-v1_train_04508,CBA의 실패 후에 토머스는 2000년부터 2003년까지 래리 버드의 뒤를 이어 인디애나 페이서스의 감독이 되었다.,"{'word': '인디애나 페이서스', 'start_idx': 45, 'end_idx': 53, 'type': 'ORG'}","{'word': '래리 버드', 'start_idx': 32, 'end_idx': 36, 'type': 'PER'}",no_relation(0),wikipedia
7,klue-re-v1_train_27100,"성씨인 마 씨는 예언자 무함마드의 후손이라는 것을 나타내는 것이고, 아버지의 이름 합지도 이슬람교의 성지 메카를 순례한 사람에게 붙이는 존칭인 하지에서 유래되었다.","{'word': '무함마드', 'start_idx': 13, 'end_idx': 16, 'type': 'PER'}","{'word': '이슬람교', 'start_idx': 50, 'end_idx': 53, 'type': 'ORG'}",no_relation(0),wikipedia
8,klue-re-v1_train_12205,"김명중 사장은 지난 13일 EBS 뉴스에 영상으로 출연해 ""EBS를 믿고 사랑해주신 시청자 여러분께 큰 실망을 드려 대단히 죄송하다""며 고개를 숙였다.","{'word': 'EBS', 'start_idx': 33, 'end_idx': 35, 'type': 'ORG'}","{'word': '김명중', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}",org:top_members/employees(10),wikitree
9,klue-re-v1_train_14933,호날두는 2003년 8월 스포르팅 리스본이 맨체스터 유나이티드를 상대로 한 이스타디우 주제 알발라드 개장 경기에서 3-1로 이기는 것을 목격한 맨체스터 유나이티드의 알렉스 퍼거슨 감독의 눈길을 끌었다.,"{'word': '알렉스 퍼거슨', 'start_idx': 92, 'end_idx': 98, 'type': 'PER'}","{'word': '맨체스터 유나이티드', 'start_idx': 80, 'end_idx': 89, 'type': 'ORG'}",per:employee_of(18),wikipedia


## 훈련 / 테스트 데이터 세트 분리

In [ ]:
df_train = pd.DataFrame(raw_datasets["train"][train_id])
df_valid = pd.DataFrame(raw_datasets["validation"][valid_id])

# Tag 삽입

In [ ]:
"""
    텍스트 내 주어/목적어 개체를 식별할 수 있도록 개체 마커를 추가하는 함수입니다.

    Args:
        text: 원본 문장 (예: "John met with Mary")
        subject_range: 주어 개체의 시작 및 끝 인덱스 (예: (0, 3) -> "John")
        object_range: 목적어 개체의 시작 및 끝 인덱스 (예: (14, 17) -> "Mary")

    Returns:
        주어/목적어 개체 마커가 포함된 문자열
    """

    # 주어의 위치가 목적어의 위치보다 앞에 있을 때
    if subject_range < object_range:
        segments = [
            text[: subject_range[0]],  # 주어 개체 전의 텍스트
            subject_start_marker,  # 주어 시작 마커 (예: "[SUBJ]")
            text[subject_range[0] : subject_range[1] + 1],  # 주어 개체 텍스트
            subject_end_marker,  # 주어 끝 마커 (예: "[/SUBJ]")
            text[subject_range[1] + 1 : object_range[0]],  # 주어와 목적어 사이의 텍스트
            object_start_marker,  # 목적어 시작 마커 (예: "[OBJ]")
            text[object_range[0] : object_range[1] + 1],  # 목적어 개체 텍스트
            object_end_marker,  # 목적어 끝 마커 (예: "[/OBJ]")
            text[object_range[1] + 1 :],  # 목적어 개체 뒤의 텍스트
        ]

    # 목적어의 위치가 주어의 위치보다 앞에 있을 때
    elif subject_range > object_range:
        segments = [
            text[: object_range[0]],  # 목적어 개체 전의 텍스트
            object_start_marker,  # 목적어 시작 마커 (예: "[OBJ]")
            text[object_range[0] : object_range[1] + 1],  # 목적어 개체 텍스트
            object_end_marker,  # 목적어 끝 마커 (예: "[/OBJ]")
            text[object_range[1] + 1 : subject_range[0]],  # 목적어와 주어 사이의 텍스트
            subject_start_marker,  # 주어 시작 마커 (예: "[SUBJ]")
            text[subject_range[0] : subject_range[1] + 1],  # 주어 개체 텍스트
            subject_end_marker,  # 주어 끝 마커 (예: "[/SUBJ]")
            text[subject_range[1] + 1 :],  # 주어 개체 뒤의 텍스트
        ]

    # 주어와 목적어의 위치가 겹칠 때 예외 처리
    else:
        raise ValueError("Entity boundaries overlap.")  # 주어와 목적어의 위치가 겹칠 경우 오류 발생

    # 모든 부분을 결합하여 최종 텍스트 생성
    marked_text = "".join(segments)

    return marked_text  # 마커가 포함된 텍스트 반환

In [ ]:
def add_tag(dframe):
    for i in dframe.index:
        dframe.loc[i, "subject_entity"]
        subject_range = (
            int(dframe.loc[i, "subject_entity"]["start_idx"]),
            int(dframe.loc[i, "subject_entity"]["end_idx"]),
        )
        object_range = (
            int(dframe.loc[i, "object_entity"]["start_idx"]),
            int(dframe.loc[i, "object_entity"]["end_idx"]),
        )
        sent = dframe.loc[i, "sentence"]
        marked_sent = mark_entity_spans(sent, subject_range, object_range)
        dframe.loc[i, "sentence"] = marked_sent
        dframe.loc[i, "label"] -= 1

In [ ]:
add_tag(df_valid)
add_tag(df_train)

In [ ]:
df_valid

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_dev_00006,<subj>심은주</subj> <obj>하나금융투자</obj> 연구원은 “매일유업의...,"{'word': '심은주', 'start_idx': 0, 'end_idx': 2, ...","{'word': '하나금융투자', 'start_idx': 4, 'end_idx': ...",17,wikitree
1,klue-re-v1_dev_00007,공개된 영상은 <obj>한국</obj> 경제의 심장부에 서 있는 채이헌 <subj>...,"{'word': '허재', 'start_idx': 29, 'end_idx': 30,...","{'word': '한국', 'start_idx': 8, 'end_idx': 9, '...",16,wikitree
2,klue-re-v1_dev_00009,<obj>김진우</obj> <subj>한국투자증권</subj> 연구원은 “8년 만에...,"{'word': '한국투자증권', 'start_idx': 4, 'end_idx': ...","{'word': '김진우', 'start_idx': 0, 'end_idx': 2, ...",9,wikitree
3,klue-re-v1_dev_00010,<subj>포천시</subj> <obj>관계자</obj>는 “현장 중심의 공정하고 ...,"{'word': '포천시', 'start_idx': 0, 'end_idx': 2, ...","{'word': '관계자', 'start_idx': 4, 'end_idx': 6, ...",9,wikitree
4,klue-re-v1_dev_00012,환경오염 배출시설 및 방지시설 현장 공개 방안에 대해서는 <subj>전라남도</su...,"{'word': '전라남도', 'start_idx': 32, 'end_idx': 3...","{'word': '여수시', 'start_idx': 38, 'end_idx': 40...",5,wikitree
...,...,...,...,...,...,...
3129,klue-re-v1_dev_07755,<subj>한신대학교</subj>는 1940년 <obj>한국</obj> 최초의 신학...,"{'word': '한신대학교', 'start_idx': 0, 'end_idx': 4...","{'word': '한국', 'start_idx': 13, 'end_idx': 14,...",2,wikitree
3130,klue-re-v1_dev_07757,<subj>구</subj> <obj>선수</obj>는 경기 중 80% 정도는 결국 ...,"{'word': '구', 'start_idx': 0, 'end_idx': 0, 't...","{'word': '선수', 'start_idx': 2, 'end_idx': 3, '...",28,wikitree
3131,klue-re-v1_dev_07758,래퍼 <subj>뱃사공</subj>(<obj>김진우</obj>·33)이 11일 인스...,"{'word': '뱃사공', 'start_idx': 3, 'end_idx': 5, ...","{'word': '김진우', 'start_idx': 7, 'end_idx': 9, ...",19,wikitree
3132,klue-re-v1_dev_07762,그러므로 동전의 변덕스러운 회전들은 2명의 예민한 거인 - 7 승 0 패의 <sub...,"{'word': '올라주원', 'start_idx': 42, 'end_idx': 4...","{'word': '샘슨', 'start_idx': 57, 'end_idx': 58,...",25,wikipedia


In [ ]:
df_train

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_train_00002,K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",4,wikitree
1,klue-re-v1_train_00003,균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</o...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",9,wikitree
2,klue-re-v1_train_00005,": 유엔, 유럽 의회, <subj>북대서양 조약 기구</subj> (<obj>NAT...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...",3,wikipedia
3,klue-re-v1_train_00007,"<subj>박용오</subj>(朴容旿, <obj>1937년 4월 29일</obj>(...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...",11,wikipedia
4,klue-re-v1_train_00008,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 <obj>타이베이</obj...,"{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...","{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...",2,wikipedia
...,...,...,...,...,...,...
22931,klue-re-v1_train_32464,KIA타이거즈 <obj>외야수</obj> <subj>이창진</subj>이 롯데백화점...,"{'word': '이창진', 'start_idx': 12, 'end_idx': 14...","{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...",28,wikitree
22932,klue-re-v1_train_32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",17,wikitree
22933,klue-re-v1_train_32466,"법포는 다시 <subj>최시형</subj>, 서병학, <obj>손병희</obj> 직...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",25,wikipedia
22934,klue-re-v1_train_32467,<subj>완도군</subj>(군수 <obj>신우철</obj>)이 국토교통부에서 실...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",9,wikitree


# 데이터 세트 구성

In [ ]:
import datasets
from datasets import Dataset

# Pandas 데이터 프레임의 내용을 학습에 용이하도록 딕셔너리 형식으로 변환
datasets_refined = datasets.DatasetDict(
    {
        "train": Dataset.from_pandas(df_train),
        "validation": Dataset.from_pandas(df_valid),
    }
)

# Tokenizer 설정

In [ ]:
from transformers import AutoTokenizer

# klue/roberta-base 이외의 여러 Tokenizer 및 모델을 고려할 수 있습니다. https://huggingface.co/klue
model_checkpoint = "klue/roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# special token 추가
tokenizer.add_special_tokens(
    {
        "additional_special_tokens": [
            subject_start_marker,
            subject_end_marker,
            object_start_marker,
            object_end_marker,
        ]
    }
)

4

# metric 불러오기
datasets 패키지에 각종 평가 지표(metric)이 정의되어 있습니다.

In [ ]:
from datasets import load_metric

# metric.inputs_description
metric = load_metric("f1", trust_remote_code=True)

# 인코딩

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, return_token_type_ids=False)

In [ ]:
tokenizer.convert_tokens_to_ids("</obj>")

32003

In [ ]:
len(tokenizer)

32004

In [ ]:
def preprocess_function(examples, tokenizer_type="bert-wp"):
    """
    입력 데이터를 전처리하여 모델에 적합한 형식으로 변환하는 함수입니다.

    Args:
        examples: 입력 데이터(문장 및 레이블)
        tokenizer_type: 사용할 토크나이저의 유형 ('bert-wp' 또는 'xlm-sp')

    Returns:
        모델 입력에 사용할 토큰화된 데이터
    """

    # 레이블을 인덱스로 매핑합니다. 예: { 'no_relation': 1, 'org:alternate_names': 2, ... }
    label_map = {
        label: i + 1 for i, label in enumerate(label_names)
    }  # 레이블 인덱스가 1부터 시작하도록 설정
    labels = examples["label"]  # 데이터셋의 레이블 정보를 가져옵니다.

    def fix_tokenization_error(text: str, tokenizer_type: str) -> Any:
        """
        개체 마커가 단어 중간에 삽입된 경우 발생할 수 있는 토크나이제이션 오류를 수정하는 함수입니다.

        Args:
            text: 마커가 포함된 원본 텍스트 (예: "<obj>조지 해리슨</obj>이 <subj>비틀즈</subj>가")
            tokenizer_type: 사용할 토크나이저 유형 ('bert-wp' 또는 'xlm-sp')

        Returns:
            수정된 토큰 리스트
        """
        tokens = tokenizer.tokenize(text)  # 주어진 텍스트를 토큰화합니다.

        # 주어(subj) 마커 다음에 공백이 없을 경우 처리
        if text[text.find(subject_end_marker) + len(subject_end_marker)] != " ":
            space_idx = (
                tokens.index(subject_end_marker) + 1
            )  # 마커 바로 다음의 토큰 인덱스를 찾습니다.
            if tokenizer_type == "xlm-sp":
                # xlm-sp 토크나이저의 경우, 추가된 언더바(▁)를 제거합니다.
                if tokens[space_idx] == "▁":
                    tokens.pop(space_idx)
                elif tokens[space_idx].startswith("▁"):
                    tokens[space_idx] = tokens[space_idx][1:]
            elif tokenizer_type == "bert-wp":
                # bert-wp 토크나이저의 경우, 해당 토큰 앞에 "##"를 추가하여 결합을 표시합니다.
                if (
                    not tokens[space_idx].startswith("##")
                    and "가" <= tokens[space_idx][0] <= "힣"
                ):
                    tokens[space_idx] = "##" + tokens[space_idx]

        # 목적어(obj) 마커 다음에 공백이 없을 경우 처리
        if text[text.find(object_end_marker) + len(object_end_marker)] != " ":
            space_idx = (
                tokens.index(object_end_marker) + 1
            )  # 마커 바로 다음의 토큰 인덱스를 찾습니다.
            if tokenizer_type == "xlm-sp":
                # xlm-sp 토크나이저의 경우, 추가된 언더바(▁)를 제거합니다.
                if tokens[space_idx] == "▁":
                    tokens.pop(space_idx)
                elif tokens[space_idx].startswith("▁"):
                    tokens[space_idx] = tokens[space_idx][1:]
            elif tokenizer_type == "bert-wp":
                # bert-wp 토크나이저의 경우, 해당 토큰 앞에 "##"를 추가하여 결합을 표시합니다.
                if (
                    not tokens[space_idx].startswith("##")
                    and "가" <= tokens[space_idx][0] <= "힣"
                ):
                    tokens[space_idx] = "##" + tokens[space_idx]

        return tokens  # 수정된 토큰 리스트를 반환합니다.

    # 각 문장에 대해 토크나이제이션 오류를 수정한 토큰 리스트를 생성합니다.
    tokenized_examples = [
        fix_tokenization_error(text, tokenizer_type) for text in examples["sentence"]
    ]

    # 주어진 토큰 리스트를 토크나이저가 사용할 수 있는 형식으로 변환합니다.
    batch_encoding = tokenizer.batch_encode_plus(
        [
            (tokenizer.convert_tokens_to_ids(list(tokens)), None)
            for tokens in tokenized_examples
        ],
        truncation=True,  # 길이가 너무 긴 문장은 자릅니다.
        return_token_type_ids=False,  # 토큰 유형 ID는 반환하지 않도록 설정합니다.
    )

    return batch_encoding  # 최종 토큰화된 데이터 반환

In [ ]:
datasets_refined["train"][:5]

{'guid': ['klue-re-v1_train_00002',
  'klue-re-v1_train_00003',
  'klue-re-v1_train_00005',
  'klue-re-v1_train_00007',
  'klue-re-v1_train_00008'],
 'sentence': ['K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일 <obj>한국프로축구연맹</obj>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
  '균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</obj>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
  ': 유엔, 유럽 의회, <subj>북대서양 조약 기구</subj> (<obj>NATO</obj>), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.',
  '<subj>박용오</subj>(朴容旿, <obj>1937년 4월 29일</obj>(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.',
  '중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 <obj>타이베이</obj>로 수도를 옮기는 것을 결정해, 남아있는 <subj>중화민국</subj>군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.'],
 'subject_entity': [{'end_idx': 24,
   'start_idx': 21,
   'type': 'ORG',
   'word': '광주FC'},
  {'end_idx': 17, 'start_idx': 13, 'type': 'O

In [ ]:
preprocess_function(datasets_refined["train"][:5])

{'input_ids': [[0, 47, 17665, 2302, 27135, 4610, 21, 2090, 2138, 4214, 2088, 1513, 2259, 32000, 4104, 10904, 32001, 2259, 3625, 4210, 2210, 32002, 3629, 17287, 20212, 32003, 3, 8862, 4415, 4422, 2522, 4852, 4422, 2138, 6157, 2227, 114, 1872, 14198, 2290, 115, 604, 114, 6646, 14198, 2290, 115, 1498, 4812, 2371, 2062, 18, 2], [0, 23306, 2116, 3799, 18319, 2532, 12, 1564, 13, 32000, 27930, 24393, 2024, 32001, 12, 3661, 32002, 6580, 2144, 32003, 13, 793, 1726, 11235, 22328, 8151, 2200, 5117, 2069, 585, 2088, 1513, 2259, 3900, 16955, 2170, 4202, 13473, 2138, 4535, 2371, 4683, 3633, 2210, 3705, 2062, 18, 2], [0, 30, 6125, 16, 4227, 4570, 16, 32000, 23483, 2112, 2221, 8604, 5255, 32001, 12, 32002, 19552, 11216, 32003, 13, 16, 3854, 2052, 2223, 11181, 16, 3665, 5308, 5255, 12, 21534, 13, 16, 16070, 4637, 16, 7814, 4203, 5255, 16, 4227, 5401, 4203, 5255, 16, 3854, 5071, 6898, 16, 28996, 11181, 3673, 4377, 2258, 2208, 2209, 18, 2], [0, 32000, 12365, 2168, 32001, 12, 393, 3, 3, 16, 32002, 20533, 

In [ ]:
ex = [e["label"] for e in datasets_refined["train"]]
print(ex[:2])

[4, 9]


In [ ]:
datasets_refined["train"]["sentence"]

['K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일 <obj>한국프로축구연맹</obj>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.',
 '균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</obj>)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.',
 ': 유엔, 유럽 의회, <subj>북대서양 조약 기구</subj> (<obj>NATO</obj>), 국제이주기구, 세계 보건 기구 (WHO), 지중해 연합, 이슬람 협력 기구, 유럽 안보 협력 기구, 국제 통화 기금, 세계무역기구 그리고 프랑코포니.',
 '<subj>박용오</subj>(朴容旿, <obj>1937년 4월 29일</obj>(음력 3월 19일)(음력 3월 19일) ~ 2009년 11월 4일)는 서울에서 태어난 대한민국의 기업인으로 두산그룹 회장, KBO 총재 등을 역임했다.',
 '중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 <obj>타이베이</obj>로 수도를 옮기는 것을 결정해, 남아있는 <subj>중화민국</subj>군의 병력이나 국가, 개인의 재산등을 속속 타이완으로 옮기기 시작해, 12월에는 중앙 정부 기구도 모두 이전해 타이베이 시를 중화민국의 새로운 수도로 삼았다.',
 '특히 김동연 전 경제부총리를 비롯한 김두관 국회의원, <subj>안규백</subj> 국회의원, 김종민 국회의원, 오제세 국회의원, 최운열 국회의원, 김정우 국회의원, 권칠승 국회의원, 맹성규 국회의원등 <obj>더불어민주당</obj> 국회의원 8명이 영상 축하 메세지를 보내 눈길을 끌었다.',
 '<subj>하비에르 파스토레</subj>는 <obj>아르헨티나</obj> 클럽 타예레스의 유소년팀에서 축구를 시작하였다.',
 "이른바 'Z세대'로 불리는 1990년대 중반 이후 태어난 세대에게 대표 아이콘으로 통하는 미국 <obj>싱어송라이터<

In [ ]:
encoded_datasets = datasets_refined.map(preprocess_function, batched=True)

Map:   0%|          | 0/22936 [00:00<?, ? examples/s]

Map:   0%|          | 0/3134 [00:00<?, ? examples/s]

In [ ]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source', 'input_ids', 'attention_mask'],
        num_rows: 22936
    })
    validation: Dataset({
        features: ['guid', 'sentence', 'subject_entity', 'object_entity', 'label', 'source', 'input_ids', 'attention_mask'],
        num_rows: 3134
    })
})

# 모델 정의

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 29
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 토큰 임베딩 개수 늘려주기. special token을 임의로 추가했으므로
model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 768, padding_idx=1)

# 훈련

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    """
    평가 예측값과 실제 레이블을 바탕으로 메트릭을 계산하는 함수입니다.

    Args:
        eval_pred: 튜플 (predictions, labels)
            - predictions: 모델이 예측한 값, 각 클래스에 대한 확률을 포함한 배열
            - labels: 실제 레이블 값

    Returns:
        계산된 메트릭 값 (예: 정확도, F1 점수 등)
    """

    predictions, labels = eval_pred  # 예측값과 레이블을 튜플에서 분리합니다.

    # 예측된 확률값에서 가장 높은 값을 가진 클래스의 인덱스를 선택합니다.
    # 예를 들어, 각 클래스에 대한 확률이 [0.1, 0.7, 0.2]라면, argmax는 1을 반환합니다.
    predictions = np.argmax(predictions, axis=1)

    # 지정된 메트릭을 계산합니다. 여기서 'micro' 평균을 사용합니다.
    # 'micro' 평균은 전체 데이터셋에 대해 TP, FP, FN을 합산하여 평가하는 방법입니다.
    return metric.compute(predictions=predictions, references=labels, average="micro")

In [ ]:
from transformers import TrainingArguments

batch_size = 16
metric_name = "f1"

args = TrainingArguments(
    "klue-re",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

# 대략 10분 정도 걸림. A100 gpu 기준
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.617300,0.810246,0.774729
2,0.336000,0.684190,0.828015
3,0.220700,0.716590,0.831525
4,0.163500,0.747968,0.839502
5,0.115800,0.798230,0.842693


TrainOutput(global_step=7170, training_loss=0.3418406324240287, metrics={'train_runtime': 650.1324, 'train_samples_per_second': 176.395, 'train_steps_per_second': 11.029, 'total_flos': 6503765785598496.0, 'train_loss': 0.3418406324240287, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7982299327850342,
 'eval_f1': 0.8426930440331845,
 'eval_runtime': 5.1809,
 'eval_samples_per_second': 604.917,
 'eval_steps_per_second': 37.831,
 'epoch': 5.0}

# 파이프라인 구축

In [ ]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="/content/klue-re/checkpoint-7170",
    return_all_scores=True,
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
question = "<subj>이순신</subj>은 <obj>1545년</obj>에 태어났다"
classifier(question)

[[{'label': 'LABEL_0', 'score': 4.451775748748332e-05},
  {'label': 'LABEL_1', 'score': 7.870927220210433e-05},
  {'label': 'LABEL_2', 'score': 3.3007341698976234e-05},
  {'label': 'LABEL_3', 'score': 2.5670526156318374e-05},
  {'label': 'LABEL_4', 'score': 1.1598053788475227e-05},
  {'label': 'LABEL_5', 'score': 1.953785977093503e-05},
  {'label': 'LABEL_6', 'score': 2.343590858799871e-05},
  {'label': 'LABEL_7', 'score': 3.074956839554943e-05},
  {'label': 'LABEL_8', 'score': 9.224104360328056e-06},
  {'label': 'LABEL_9', 'score': 7.037444447632879e-05},
  {'label': 'LABEL_10', 'score': 2.3811213395674713e-05},
  {'label': 'LABEL_11', 'score': 0.9991310238838196},
  {'label': 'LABEL_12', 'score': 9.013551607495174e-05},
  {'label': 'LABEL_13', 'score': 4.2397026845719665e-05},
  {'label': 'LABEL_14', 'score': 1.9279876141808927e-05},
  {'label': 'LABEL_15', 'score': 1.9693350623128936e-05},
  {'label': 'LABEL_16', 'score': 2.4806093279039487e-05},
  {'label': 'LABEL_17', 'score': 1.7

In [ ]:
subject_start_marker = "<subj>"
subject_end_marker = "</subj>"
object_start_marker = "<obj>"
object_end_marker = "</obj>"

In [ ]:
label_names[11]

'per:date_of_birth'

In [ ]:
question = "<subj>소민호</subj>는 <obj>1988년</obj>에 태어났다"
classifier(question)

[[{'label': 'LABEL_0', 'score': 4.507194898906164e-05},
  {'label': 'LABEL_1', 'score': 8.391249139094725e-05},
  {'label': 'LABEL_2', 'score': 3.3824264392023906e-05},
  {'label': 'LABEL_3', 'score': 2.566513830970507e-05},
  {'label': 'LABEL_4', 'score': 1.1794085367000662e-05},
  {'label': 'LABEL_5', 'score': 2.066504384856671e-05},
  {'label': 'LABEL_6', 'score': 2.4087206838885322e-05},
  {'label': 'LABEL_7', 'score': 3.1090021366253495e-05},
  {'label': 'LABEL_8', 'score': 1.0918683983618394e-05},
  {'label': 'LABEL_9', 'score': 8.369319402845576e-05},
  {'label': 'LABEL_10', 'score': 2.407606916676741e-05},
  {'label': 'LABEL_11', 'score': 0.9991195797920227},
  {'label': 'LABEL_12', 'score': 8.558670378988609e-05},
  {'label': 'LABEL_13', 'score': 3.73018738173414e-05},
  {'label': 'LABEL_14', 'score': 1.9165847334079444e-05},
  {'label': 'LABEL_15', 'score': 1.866393540694844e-05},
  {'label': 'LABEL_16', 'score': 2.310222407686524e-05},
  {'label': 'LABEL_17', 'score': 1.7535

In [ ]:
sentence = "K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일 <obj>한국프로축구연맹</obj>으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다."
classifier(sentence)

[[{'label': 'LABEL_0', 'score': 4.61282288597431e-05},
  {'label': 'LABEL_1', 'score': 5.996734034852125e-05},
  {'label': 'LABEL_2', 'score': 0.00039574786205776036},
  {'label': 'LABEL_3', 'score': 0.0001553676265757531},
  {'label': 'LABEL_4', 'score': 0.9973824620246887},
  {'label': 'LABEL_5', 'score': 0.000279395142570138},
  {'label': 'LABEL_6', 'score': 9.351570042781532e-05},
  {'label': 'LABEL_7', 'score': 0.00018359125533606857},
  {'label': 'LABEL_8', 'score': 5.425622293842025e-05},
  {'label': 'LABEL_9', 'score': 0.0002832231402862817},
  {'label': 'LABEL_10', 'score': 5.4196749260881916e-05},
  {'label': 'LABEL_11', 'score': 3.730010212166235e-05},
  {'label': 'LABEL_12', 'score': 4.262008224031888e-05},
  {'label': 'LABEL_13', 'score': 2.737491377047263e-05},
  {'label': 'LABEL_14', 'score': 3.565421502571553e-05},
  {'label': 'LABEL_15', 'score': 5.314981171977706e-05},
  {'label': 'LABEL_16', 'score': 7.963117968756706e-05},
  {'label': 'LABEL_17', 'score': 0.00024945

In [ ]:
def convert_label_indices(predictions):
    """
    LABEL_ 형식의 인덱스를 실제 레이블 이름으로 변환하는 함수.

    Args:
        predictions: 모델 예측 결과. LABEL_인덱스와 score가 포함된 리스트.

    Returns:
        실제 레이블 이름과 score가 포함된 리스트.
    """
    converted_predictions = []
    for prediction in predictions:
        label_index = int(prediction["label"].split("_")[-1])  # LABEL_x에서 x를 추출
        label_name = label_names[label_index]  # 실제 레이블 이름으로 변환
        converted_predictions.append(
            {"label": label_name, "score": prediction["score"]}
        )

    return converted_predictions

'org:member_of'